In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import glob
import os
import pickle

%matplotlib inline

In [2]:
const = {}
const['camera_calibration'] = './camera_cal'
const['examples'] = './examples'
const['test_images'] = './test_images'
const['output'] = './output_images'
const['project_video'] = './project_video.mp4'
const['challenge_video'] = './challange_video.mp4'
const['harder_challenge_video'] = './harder_challenge_video.mp4'
const['saved_assets'] = './saved_assets'

In [3]:
def camera_calibration_parameters(imgs, nx, ny, plot=False):
    imgpoints = [] # 2D points in image
    objpoints = [] # 3D points in real world

    objp = np.zeros((nx * ny, 3), np.float32)
    objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)

    ret = False
    c_mtx, dist = None, None
    for img_path in calibration_img_paths:
        img = mpimg.imread(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

        # Find the corners
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

        # if corners are found, add object and image points
        if ret == True:
            imgpoints.append(corners)
            objpoints.append(objp) # all object points will be the same for each image
            filename = img_path.split('/')[-1]
            print('Cornors found for image: {}'.format(filename))
            
            
            # Draw and display corners
            if plot:
                img = cv2.drawChessboardCorners(img, (nx, ny), corners, ret)
                plt.imshow(img)
                plt.show()

            
    ret, c_mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    return ret, c_mtx, dist

calibration_img_paths = glob.glob(os.path.join(const['camera_calibration'], '*.jpg'))
calibration_params = camera_calibration_parameters(calibration_img_paths, 9, 6, False)

Cornors found for image: calibration13.jpg
Cornors found for image: calibration15.jpg
Cornors found for image: calibration19.jpg
Cornors found for image: calibration12.jpg
Cornors found for image: calibration9.jpg
Cornors found for image: calibration20.jpg
Cornors found for image: calibration14.jpg
Cornors found for image: calibration7.jpg
Cornors found for image: calibration6.jpg
Cornors found for image: calibration10.jpg
Cornors found for image: calibration2.jpg
Cornors found for image: calibration3.jpg
Cornors found for image: calibration8.jpg
Cornors found for image: calibration17.jpg
Cornors found for image: calibration18.jpg
Cornors found for image: calibration16.jpg
Cornors found for image: calibration11.jpg


In [3]:
def save_calibration_params(calibration_params):
    loc = os.path.join(const['saved_assets'], 'calibration_params.p')
    pickle.dump(calibration_params, open( loc, "wb" ) )
    print('saved asset to:', loc)

def get_calibration_params(loc):
    calibration_params = pickle.load( open( loc, "rb" ) )
    return calibration_params
#save_calibration_params(calibration_params)
calibration_params  = get_calibration_params('./saved_assets/calibration_params.p')

In [14]:
def undistort(img):
    ret, mtx, dist = calibration_params
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

def undistort_images(images, save=False, plot=True):
    for idx, img in enumerate(images):
        filename = img.split('/')[-1]
        output_path = os.path.join(const['output'], filename)

        
        img = mpimg.imread(img)
        undist = undistort(img)

        if save:
            mpimg.imsave(output_path, undist)
            print('saved image:', output_path )
        
        if plot:
            f, ax = plt.subplots(1, 2, figsize=(20, 20))
            ax[0].imshow(img)
            ax[0].set_title('Original image', fontsize=15)
            ax[1].imshow(undist)
            ax[1].set_title('Undistorted image', fontsize=15)
            ax[0].axis('off')
            ax[1].axis('off')
            plt.show()
    

test_imgs = glob.glob(os.path.join(const['test_images'], '*.jpg'))
undistort_images(test_imgs, save=True, plot=False)

saved image: ./output_images/test1.jpg
saved image: ./output_images/straight_lines2.jpg
saved image: ./output_images/test3.jpg
saved image: ./output_images/test4.jpg
saved image: ./output_images/test5.jpg
saved image: ./output_images/test2.jpg
saved image: ./output_images/straight_lines1.jpg
saved image: ./output_images/test6.jpg
